In [3]:
from splinter import Browser
from splinter.element_list import ElementList
from splinter.driver import ElementAPI
from selenium.webdriver import ChromeOptions

In [139]:
# Init Settings
rancher_settings: dict = {
    'url': 'https://rancher.fabrica.local/',
    'username': r'',
    'password': '',
    'cluster': '',
    'projects': [ ]
}

In [140]:
# Chrome Driver Settings
options: ChromeOptions = ChromeOptions()
options.add_argument('--start-maximized')
settings: dict = {
    'incognito': True,
    'headless': False,
    'options': options,
    'executable_path': 'chromedriver'
}

In [141]:
# Init Browser
browser: Browser = None

try:
    browser = Browser("chrome", **settings)
except SessionNotCreatedException as error:
    raise Exception(
        "R2MD couldn't start ChromeDriver properly! "
        "Make sure you are using right version. For "
        "more details consider to visit "
        "https://chromedriver.chromium.org/downloads "
        "and download a valid driver compatible with "
        "your Chrome version.\n"
        f"- Error: {error}")

In [142]:
# Access page
browser.visit(rancher_settings.get('url'))

In [144]:
def __get_url(destination: str) -> str:
    base_url:str = rancher_settings.get('url')
    if not destination.startswith('/'):
        destination = f'/{destination}'
    
    return f'{base_url}{destination}'

def __fetch_and_click_by_id(browser: Browser, item_id: str, **kwargs: dict) -> bool:
    id_list: ElementList = browser.find_by_id(item_id)
    if id_list.is_empty():
        error: str = f"Unable to find any element with ID '{item_id}'!"
        
        if kwargs.get('raise_exception'):
            raise Exception(error)
        else:
            print(error)

        return False
        
    if kwargs.get('multiple'):
        for i in range(len(id_list)):
            id_item: ElementAPI = id_list[i]
            id_item.click()
    else:
        first_id_item: ElementAPI = id_list.first
        first_id_item.click()

    return True

def __fill_by_id(browser: Browser, item_id: str, value: str, **kwargs: dict) -> None:
    id_list: ElementList = browser.find_by_id(item_id)
    if id_list.is_empty():
        error: str = f"Unable to find any element with ID '{item_id}'!"
        
        if kwargs.get('raise_exception'):
            raise Exception(error)
        else:
            print(error)

        return

    if kwargs.get('multiple'):
        for i in range(len(id_list)):
            id_item: ElementAPI = id_list[i]
            id_item.fill(value)
    else:
        first_id_item: ElementAPI = id_list.first
        first_id_item.fill(value)

def __perform_login(browser: Browser) -> None:
    credentials: list = [
        {
            'item_id': 'login-username-activedirectory',
            'value': rancher_settings.get('username'),
            'raise_exception': True
        },
        {
            'item_id': 'login-password-activedirectory',
            'value': rancher_settings.get('password'),
            'raise_exception': True
        }
    ]

    for credential in credentials:
        credential: dict = credential
        __fill_by_id(browser, **credential)

    login_form: ElementList = browser.find_by_css('form[class="text-left mt-10"]')
    if login_form.is_empty():
        raise Exception("Login form not found!")

    login_form_btn: ElementList = login_form.first.find_by_css('p button')
    if login_form_btn.is_empty():
        raise Exception("Login form button not found!")

    login_form_btn: ElementAPI = login_form_btn.first
    login_form_btn.click()

    error_banner: ElementList = browser.find_by_css('section div[class="banner p-10 bg-error"]')
    if not error_banner.is_empty():
        error: str = error_banner.first.text
        login_url: str = __get_url('/login')

        print(f"Redirecting to login page -> {base_url}")
        browser.visit(login_url)
        raise Exception(error)

#def __get_project_info(browser: Browser, cluster: str, project: str) -> dict:


In [145]:
if __fetch_and_click_by_id(browser, 'details-button', raise_exception=True):
    __fetch_and_click_by_id(browser, 'proceed-link', raise_exception=True)

In [146]:
# Performs login to Rancher
__perform_login(browser)

In [147]:
# Navigate through cluster
cluster_url: str = __get_url('/g/clusters')
browser.visit(cluster_url)

In [148]:
# Fetch for target cluster
cluster_name: str = rancher_settings.get('cluster')
clusters: ElementList = browser.find_by_css('main[class="clearfix"] div table tbody tr')
if clusters.is_empty():
    raise Exception('There is no cluster yet.')

for i in range(len(clusters)):
    cluster_node: ElementAPI = clusters[i]
    cluster: ElementAPI = cluster_node.find_by_css('td[data-title="Cluster Name: "] a')
    if cluster.text != cluster_name:
        continue

    cluster_node.find_by_css('td[class="actions"] div div[role="button"]').first.click()
    cluster_actions: ElementList = browser.find_by_css('body div[id="ember-basic-dropdown-wormhole"] div li a')
    for j in range(len(cluster_actions)):
        cluster_action: ElementAPI = cluster_actions[j]
        span_title: ElementList = cluster_action.find_by_css('span[class="pull-left"]')
        if span_title.is_empty() or span_title.first.text != "View in API":
            continue

        cluster_action.click()
        break
    
    break

In [155]:
# Navigate through projects
browser.windows[1].is_current = True
browser.click_link_by_partial_href('/projects')

### TODO
I need to navigate through projects JSON data and get their relative URL.

In [ ]:
# Fetch for project info
projects: list = rancher_settings.get('projects')
projects_info: list = []
for project in projects:
    project: str = project
    project_info: dict = __get_project_info(browser, cluster, project)
    projects_info.append(project_info)

In [2]:
# Close browser
browser.quit()

NameError: name 'browser' is not defined